In [1]:
import pandas as pd
import numpy as np
import optuna

In [2]:
train = pd.read_csv("data/train.csv")
train.head()

,id,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Age
0,0,I,1.5250,1.1750,0.3750,28.973189,12.728926,6.647958,8.348928,9
1,1,I,1.1000,0.8250,0.2750,10.418441,4.521745,2.324659,3.401940,8
2,2,M,1.3875,1.1125,0.3750,24.777463,11.339800,5.556502,6.662133,9
3,3,F,1.7000,1.4125,0.5000,50.660556,20.354941,10.991839,14.996885,11
4,4,I,1.2500,1.0125,0.3375,23.289114,11.977664,4.507570,5.953395,8


In [27]:
feature_cols = ['Sex','Length','Diameter','Height','Weight','Shucked Weight','Viscera Weight','Shell Weight']
X = train[feature_cols]
y = train.Age

In [28]:
def create_feature(df):
#     df['bmi'] = df['Height']/df['Weight'] #可加
    
#     df['area'] = df['Length'] * (df['Diameter']/2) *  np.pi #可加
    
#     #df['Shucked_Viscera'] = (df['Shucked Weight'] / df['Viscera Weight'])/df['Shell Weight']
    
#     df['Weight_Shucked'] = df['Weight']/df['Shucked Weight']
    
#     df['Weight_Viscera'] = df['Weight']/df['Viscera Weight']
    
#     df['Weight_Shell'] = df['Weight']/df['Shell Weight'] #没用
    
#     df['Height_Shucked'] = df['Height']/df['Shucked Weight']
    
#     #df['Height_Viscera'] = df['Height']/df['Viscera Weight']
    
#     #df['Height_Shell'] = df['Height']/df['Shell Weight']

    df['Total Weight'] = df['Weight']
    df['Viscera Ratio'] = df['Viscera Weight'] / df['Total Weight']
    df['Shell Ratio'] = df['Shell Weight'] / df['Total Weight']
    df['Shell-to-Body Ratio'] = df['Shell Weight'] / (df['Total Weight'] + df['Shell Weight'])
    df['Meat Yield'] = df['Shucked Weight'] / (df['Total Weight'] + df['Shell Weight'])
    df['Length-to-Diameter Ratio'] = df['Length'] / df['Diameter']
    df['Weight-to-VisceraWeight Ratio'] = df['Total Weight'] / df['Viscera Weight']
    df['Weight-to-ShellWeight Ratio'] = df['Total Weight'] / df['Shell Weight']
    df['Weight-to-ShuckedWeight Ratio']= df['Total Weight'] / df['Shucked Weight']
    
    df['Surface Area'] = 2 * (df['Length'] * df['Diameter'] + df['Length'] * df['Height'] + df['Diameter'] * df['Height'])
#    df['Volume'] = df['Length'] * df['Diameter'] * df['Height']
#    df['Density'] = df['Total Weight'] / df['Volume']
#    df['Pseudo BMI'] = df['Total Weight'] / (df['Height'] ** 2)
    
    df['Length^2'] = df['Length'] ** 2
    df['Diameter^2'] = df['Diameter'] ** 2
    
    df['Log Weight'] = np.log(df['Total Weight'] + 1)
    
    df['Weight_wo_Viscera'] = df['Shucked Weight'] - df['Viscera Weight']
    df['Body Condition Index'] = np.sqrt(df['Length'] * df['Total Weight'] * df['Shucked Weight'])
        
    return df

In [29]:
X = create_feature(X)
X.drop(columns=['Total Weight'],inplace=True)
X.head()

C:\Users\lijin\AppData\Local\Temp\ipykernel_25016\3943852568.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Total Weight'] = df['Weight']


,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Viscera Ratio,Shell Ratio,...,Length-to-Diameter Ratio,Weight-to-VisceraWeight Ratio,Weight-to-ShellWeight Ratio,Weight-to-ShuckedWeight Ratio,Surface Area,Length^2,Diameter^2,Log Weight,Weight_wo_Viscera,Body Condition Index
0,I,1.5250,1.1750,0.3750,28.973189,12.728926,6.647958,8.348928,0.229452,0.288160,...,1.297872,4.358209,3.470289,2.276169,5.608750,2.325625,1.380625,3.400303,6.080968,23.715318
1,I,1.1000,0.8250,0.2750,10.418441,4.521745,2.324659,3.401940,0.223129,0.326531,...,1.333333,4.481707,3.062500,2.304075,2.873750,1.210000,0.680625,2.435230,2.197086,7.198645
2,M,1.3875,1.1125,0.3750,24.777463,11.339800,5.556502,6.662133,0.224256,0.268879,...,1.247191,4.459184,3.719149,2.185000,4.962188,1.925156,1.237656,3.249501,5.783298,19.744567
3,F,1.7000,1.4125,0.5000,50.660556,20.354941,10.991839,14.996885,0.216970,0.296027,...,1.203540,4.608925,3.378072,2.488858,7.915000,2.890000,1.995156,3.944695,9.363103,41.869171
4,I,1.2500,1.0125,0.3375,23.289114,11.977664,4.507570,5.953395,0.193548,0.255630,...,1.234568,5.166667,3.911905,1.944379,4.058438,1.562500,1.025156,3.190028,7.470093,18.673149


In [30]:
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import VotingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.compose import make_column_selector,make_column_transformer
import lightgbm as lgb
from catboost import CatBoostRegressor
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [33]:
def objective(trial):
    # 定义参数空间
    lgbm_params = {
        'boosting_type': trial.suggest_categorical('boosting_type', ['gbdt', 'dart']),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 500, 2000),
        'num_leaves': trial.suggest_int('num_leaves', 16, 64),
        'random_state': 318
    }

    cbr_params = {
        'random_state': 318,
        'objective': 'MAE',
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'depth': trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 0.1, 10.0)
    }

    hgbr_params = {
        'random_state': 318,
        'loss': 'absolute_error'
    }
    
    weight_lgbm = trial.suggest_int('weight_lgbm', 1, 2)
    weight_cbr = trial.suggest_int('weight_cbr', 1, 2)
    weight_hgbr = trial.suggest_int('weight_hgbr', 1, 2)
    
    num_cols = make_column_selector(dtype_include='number')
    cat_cols = make_column_selector(dtype_exclude='number')
    
    # 定义预处理步骤
    preprocessor = make_column_transformer(
        (StandardScaler(), num_cols),
        (OneHotEncoder(handle_unknown='ignore'), cat_cols)
    )

    # 定义模型和管道
    lgbm = lgb.LGBMRegressor(**lgbm_params)
    cbr = CatBoostRegressor(**cbr_params)
    hgbr = HistGradientBoostingRegressor(**hgbr_params)

    vr = VotingRegressor([('lgbm', lgbm), ('cbr', cbr), ('hgbr', hgbr)],weights=(weight_lgbm, weight_cbr, weight_hgbr))

    # 定义管道
    pipe = make_pipeline(preprocessor, vr)

    # 拆分数据集
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=318)

    # 拟合模型
    pipe.fit(X_train, y_train)

    # 预测验证集
    y_pred = pipe.predict(X_val)

    # 计算MAE作为目标函数
    mae = mean_absolute_error(y_val, y_pred)

    return mae

In [32]:
# 创建Optuna Study对象
study = optuna.create_study(direction='minimize')

# 运行优化
study.optimize(objective, n_trials=50)

# 打印最佳超参数配置和最佳MAE
print("Best MAE:", study.best_value)
print("Best Params:", study.best_params)

[I 2023-06-10 22:25:48,049] A new study created in memory with name: no-name-d2ce2744-302a-45b9-bf4b-95e92ff35df4


0:	learn: 2.3045342	total: 155ms	remaining: 2m 34s
1:	learn: 2.2838761	total: 163ms	remaining: 1m 21s
2:	learn: 2.2626903	total: 171ms	remaining: 57s
3:	learn: 2.2417085	total: 179ms	remaining: 44.6s
4:	learn: 2.2212483	total: 186ms	remaining: 37s
5:	learn: 2.1997971	total: 195ms	remaining: 32.2s
6:	learn: 2.1795959	total: 202ms	remaining: 28.6s
7:	learn: 2.1613425	total: 210ms	remaining: 26s
8:	learn: 2.1433090	total: 219ms	remaining: 24.2s
9:	learn: 2.1234053	total: 231ms	remaining: 22.8s
10:	learn: 2.1040356	total: 239ms	remaining: 21.5s
11:	learn: 2.0858528	total: 249ms	remaining: 20.5s
12:	learn: 2.0696536	total: 259ms	remaining: 19.6s
13:	learn: 2.0530069	total: 267ms	remaining: 18.8s
14:	learn: 2.0367875	total: 276ms	remaining: 18.1s
15:	learn: 2.0209846	total: 283ms	remaining: 17.4s
16:	learn: 2.0036782	total: 291ms	remaining: 16.8s
17:	learn: 1.9872509	total: 299ms	remaining: 16.3s
18:	learn: 1.9721141	total: 306ms	remaining: 15.8s
19:	learn: 1.9584995	total: 314ms	remaining: 

168:	learn: 1.4001686	total: 1.41s	remaining: 6.91s
169:	learn: 1.3997887	total: 1.41s	remaining: 6.9s
170:	learn: 1.3993985	total: 1.42s	remaining: 6.88s
171:	learn: 1.3990470	total: 1.43s	remaining: 6.87s
172:	learn: 1.3987346	total: 1.43s	remaining: 6.85s
173:	learn: 1.3984090	total: 1.44s	remaining: 6.83s
174:	learn: 1.3981282	total: 1.45s	remaining: 6.82s
175:	learn: 1.3978694	total: 1.45s	remaining: 6.8s
176:	learn: 1.3975823	total: 1.46s	remaining: 6.78s
177:	learn: 1.3972486	total: 1.47s	remaining: 6.77s
178:	learn: 1.3969548	total: 1.47s	remaining: 6.75s
179:	learn: 1.3967129	total: 1.48s	remaining: 6.73s
180:	learn: 1.3964518	total: 1.48s	remaining: 6.71s
181:	learn: 1.3962684	total: 1.49s	remaining: 6.7s
182:	learn: 1.3959468	total: 1.5s	remaining: 6.68s
183:	learn: 1.3957277	total: 1.5s	remaining: 6.66s
184:	learn: 1.3954653	total: 1.51s	remaining: 6.65s
185:	learn: 1.3952626	total: 1.52s	remaining: 6.63s
186:	learn: 1.3949924	total: 1.52s	remaining: 6.62s
187:	learn: 1.394

345:	learn: 1.3816423	total: 2.63s	remaining: 4.98s
346:	learn: 1.3815967	total: 2.64s	remaining: 4.97s
347:	learn: 1.3815773	total: 2.65s	remaining: 4.96s
348:	learn: 1.3815519	total: 2.65s	remaining: 4.95s
349:	learn: 1.3815187	total: 2.66s	remaining: 4.94s
350:	learn: 1.3814876	total: 2.67s	remaining: 4.93s
351:	learn: 1.3814508	total: 2.67s	remaining: 4.92s
352:	learn: 1.3814251	total: 2.68s	remaining: 4.91s
353:	learn: 1.3813957	total: 2.69s	remaining: 4.9s
354:	learn: 1.3813616	total: 2.69s	remaining: 4.89s
355:	learn: 1.3813457	total: 2.7s	remaining: 4.88s
356:	learn: 1.3813210	total: 2.71s	remaining: 4.88s
357:	learn: 1.3812809	total: 2.71s	remaining: 4.86s
358:	learn: 1.3812152	total: 2.72s	remaining: 4.86s
359:	learn: 1.3811946	total: 2.73s	remaining: 4.84s
360:	learn: 1.3810923	total: 2.73s	remaining: 4.84s
361:	learn: 1.3810588	total: 2.74s	remaining: 4.83s
362:	learn: 1.3809796	total: 2.74s	remaining: 4.82s
363:	learn: 1.3809378	total: 2.75s	remaining: 4.8s
364:	learn: 1.3

504:	learn: 1.3750341	total: 3.95s	remaining: 3.87s
505:	learn: 1.3750222	total: 3.95s	remaining: 3.86s
506:	learn: 1.3749897	total: 3.96s	remaining: 3.85s
507:	learn: 1.3749781	total: 3.96s	remaining: 3.84s
508:	learn: 1.3749441	total: 3.97s	remaining: 3.83s
509:	learn: 1.3748812	total: 3.98s	remaining: 3.83s
510:	learn: 1.3748535	total: 3.99s	remaining: 3.82s
511:	learn: 1.3748126	total: 4s	remaining: 3.81s
512:	learn: 1.3748007	total: 4.01s	remaining: 3.81s
513:	learn: 1.3747566	total: 4.02s	remaining: 3.8s
514:	learn: 1.3747367	total: 4.02s	remaining: 3.79s
515:	learn: 1.3746911	total: 4.03s	remaining: 3.78s
516:	learn: 1.3746366	total: 4.04s	remaining: 3.77s
517:	learn: 1.3746075	total: 4.04s	remaining: 3.76s
518:	learn: 1.3745665	total: 4.05s	remaining: 3.75s
519:	learn: 1.3745394	total: 4.06s	remaining: 3.74s
520:	learn: 1.3745172	total: 4.07s	remaining: 3.74s
521:	learn: 1.3744836	total: 4.07s	remaining: 3.73s
522:	learn: 1.3744492	total: 4.08s	remaining: 3.72s
523:	learn: 1.37

...

516:	learn: 1.3236549	total: 5.06s	remaining: 4.73s
517:	learn: 1.3235695	total: 5.07s	remaining: 4.72s
518:	learn: 1.3235140	total: 5.08s	remaining: 4.71s
519:	learn: 1.3233818	total: 5.09s	remaining: 4.7s
520:	learn: 1.3233093	total: 5.1s	remaining: 4.69s
521:	learn: 1.3232478	total: 5.11s	remaining: 4.68s
522:	learn: 1.3232124	total: 5.12s	remaining: 4.67s
523:	learn: 1.3231887	total: 5.13s	remaining: 4.66s
524:	learn: 1.3230987	total: 5.14s	remaining: 4.65s
525:	learn: 1.3229851	total: 5.14s	remaining: 4.64s
526:	learn: 1.3228130	total: 5.15s	remaining: 4.63s
527:	learn: 1.3227796	total: 5.16s	remaining: 4.62s
528:	learn: 1.3227162	total: 5.17s	remaining: 4.6s
529:	learn: 1.3226677	total: 5.18s	remaining: 4.59s
530:	learn: 1.3226146	total: 5.19s	remaining: 4.58s
531:	learn: 1.3225413	total: 5.2s	remaining: 4.57s
532:	learn: 1.3223660	total: 5.21s	remaining: 4.56s
533:	learn: 1.3222732	total: 5.22s	remaining: 4.55s
534:	learn: 1.3222466	total: 5.23s	remaining: 4.54s
535:	learn: 1.32

688:	learn: 1.3113345	total: 6.73s	remaining: 3.04s
689:	learn: 1.3113016	total: 6.74s	remaining: 3.03s
690:	learn: 1.3112168	total: 6.75s	remaining: 3.02s
691:	learn: 1.3110948	total: 6.76s	remaining: 3.01s
692:	learn: 1.3110249	total: 6.77s	remaining: 3s
693:	learn: 1.3108601	total: 6.78s	remaining: 2.99s
694:	learn: 1.3107679	total: 6.79s	remaining: 2.98s
695:	learn: 1.3107267	total: 6.79s	remaining: 2.97s
696:	learn: 1.3105993	total: 6.8s	remaining: 2.96s
697:	learn: 1.3105733	total: 6.81s	remaining: 2.95s
698:	learn: 1.3104977	total: 6.82s	remaining: 2.94s
699:	learn: 1.3103753	total: 6.83s	remaining: 2.93s
700:	learn: 1.3102954	total: 6.84s	remaining: 2.92s
701:	learn: 1.3102702	total: 6.85s	remaining: 2.91s
702:	learn: 1.3101768	total: 6.86s	remaining: 2.9s
703:	learn: 1.3100472	total: 6.87s	remaining: 2.89s
704:	learn: 1.3098048	total: 6.88s	remaining: 2.88s
705:	learn: 1.3096984	total: 6.88s	remaining: 2.87s
706:	learn: 1.3096747	total: 6.89s	remaining: 2.86s
707:	learn: 1.309

854:	learn: 1.2995802	total: 8.38s	remaining: 1.42s
855:	learn: 1.2995556	total: 8.39s	remaining: 1.41s
856:	learn: 1.2995019	total: 8.4s	remaining: 1.4s
857:	learn: 1.2994649	total: 8.4s	remaining: 1.39s
858:	learn: 1.2994132	total: 8.41s	remaining: 1.38s
859:	learn: 1.2993708	total: 8.42s	remaining: 1.37s
860:	learn: 1.2992684	total: 8.43s	remaining: 1.36s
861:	learn: 1.2991732	total: 8.44s	remaining: 1.35s
862:	learn: 1.2990234	total: 8.45s	remaining: 1.34s
863:	learn: 1.2989915	total: 8.46s	remaining: 1.33s
864:	learn: 1.2989483	total: 8.47s	remaining: 1.32s
865:	learn: 1.2988502	total: 8.48s	remaining: 1.31s
866:	learn: 1.2988180	total: 8.49s	remaining: 1.3s
867:	learn: 1.2987303	total: 8.49s	remaining: 1.29s
868:	learn: 1.2986914	total: 8.5s	remaining: 1.28s
869:	learn: 1.2986688	total: 8.51s	remaining: 1.27s
870:	learn: 1.2986368	total: 8.52s	remaining: 1.26s
871:	learn: 1.2985786	total: 8.53s	remaining: 1.25s
872:	learn: 1.2984693	total: 8.54s	remaining: 1.24s
873:	learn: 1.298

[I 2023-06-10 23:20:24,492] Trial 49 finished with value: 1.353508538564255 and parameters: {'boosting_type': 'dart', 'learning_rate': 0.06281895152195545, 'n_estimators': 712, 'num_leaves': 56, 'depth': 6, 'l2_leaf_reg': 5.119977542926322, 'weight_lgbm': 1, 'weight_cbr': 2, 'weight_hgbr': 2}. Best is trial 49 with value: 1.353508538564255.


Best MAE: 1.353508538564255
Best Params: {'boosting_type': 'dart', 'learning_rate': 0.06281895152195545, 'n_estimators': 712, 'num_leaves': 56, 'depth': 6, 'l2_leaf_reg': 5.119977542926322, 'weight_lgbm': 1, 'weight_cbr': 2, 'weight_hgbr': 2}
